In [63]:
import numpy as np
from datascience import *
import math as m
import qgrid as q
import pandas as pd
from scipy.stats import chi2_contingency
import re

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
from scipy.stats import chi2_contingency

# Setting up Tables

In [14]:
original_data = pd.read_csv("community_RR_csv.csv")
original_data_caninervis = original_data[original_data["Syntrichia caninervis cover"] != "0%"][['Coordinates', 'Exposure of S. caninervis', 'Sex (M,F,S)']].rename({'Sex (M,F,S)':'Phenotypic Sex'}, axis=1)
original_data_caninervis

,Coordinates,Exposure of S. caninervis,Phenotypic Sex
1,T1 30,I,S
2,T1 30,I,S
3,T1 30,I,S
4,T1 30,I,S
5,T1 30,I,S
6,T1 30,I,F
7,T1 30,I,S
8,T1 30,I,S
9,T1 30,I,S
10,T1 30,I,S


In [15]:
our_data_unclean = pd.read_csv("sex_ratio_data_final.csv")
our_data_unclean.head()

,Genomics_ID,Sample_ID,loc,lat,lon,Species,Expressed Sex,Collector,Collection Year,Extraction #,PCR #,DIGEST #,Inferred Genetic Sex,NOTES,Gel Date,Lab,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,GJB341,BOWKER-T1-30,NaN,NaN,NaN,S. caninervis,U,M. Bowker,1997.0,17.0,39,24.0,F,NaN,NaN,Mishler,NaN,NaN,NaN
1,GJB342,BOWKER-T1-30,NaN,NaN,NaN,S. caninervis,U,M. Bowker,1997.0,17.0,39,24.0,F,NaN,NaN,Mishler,NaN,NaN,NaN
2,GJB343,BOWKER-T1-30,NaN,NaN,NaN,S. caninervis,U,M. Bowker,1997.0,17.0,39,24.0,F,NaN,NaN,Mishler,NaN,NaN,NaN
3,GJB344,BOWKER-T1-30,NaN,NaN,NaN,S. caninervis,U,M. Bowker,1997.0,17.0,39,24.0,F,NaN,NaN,Mishler,NaN,count F,257.0
4,GJB345,BOWKER-T1-30,NaN,NaN,NaN,S. caninervis,U,M. Bowker,1997.0,17.0,39,24.0,F,NaN,NaN,Mishler,NaN,count M,14.0


In [16]:
our_data_pre = our_data_unclean[["Sample_ID","Inferred Genetic Sex"]]
our_data = our_data_pre[~our_data_pre['Inferred Genetic Sex'].isna()].reset_index().drop('index', axis=1)
our_data.head()

,Sample_ID,Inferred Genetic Sex
0,BOWKER-T1-30,F
1,BOWKER-T1-30,F
2,BOWKER-T1-30,F
3,BOWKER-T1-30,F
4,BOWKER-T1-30,F


In [17]:
search = []    
for values in our_data['Sample_ID']:
    search.append(re.search(r'T[^.]*', values).group())

coordinates = pd.DataFrame(pd.Series(search).str.replace('-',' '))  

our_data['Coordinates'] = coordinates[0]
our_data[our_data["Coordinates"] == "T2 105"]

,Sample_ID,Inferred Genetic Sex,Coordinates
131,BOWKER-T2-105.5,F,T2 105
132,BOWKER-T2-105.6,F,T2 105
133,BOWKER-T2-105.8,F,T2 105
134,BOWKER-T2-105.9,F,T2 105
135,BOWKER-T2-105.10,M,T2 105


In [18]:
pre_gen = our_data.drop('Sample_ID', axis=1).set_index('Coordinates')
gen = pd.get_dummies(pre_gen).groupby('Coordinates').sum()
gen.head()

,Inferred Genetic Sex_F,Inferred Genetic Sex_M
Coordinates,,
T1 105,10,0
T1 135,10,0
T1 150,7,0
T1 165,7,0
T1 180,9,0


In [19]:
dumb = pd.get_dummies(original_data_caninervis[['Coordinates','Phenotypic Sex']].set_index('Coordinates'))
pre_phen = dumb.groupby('Coordinates').sum()
phen = pre_phen.join(original_data_caninervis.groupby('Coordinates').first()[['Exposure of S. caninervis']])
phen

,Phenotypic Sex_F,Phenotypic Sex_M,Phenotypic Sex_S,Exposure of S. caninervis
Coordinates,,,,
T1 105,1,0,9,I
T1 135,0,0,10,I
T1 150,0,0,10,I
T1 165,0,0,10,I
T1 180,0,0,10,I
T1 195,0,0,10,I
T1 210,0,0,10,I
T1 240,4,0,6,I
T1 30,1,0,9,I


In [20]:
df = gen.join(phen).rename({"Exposure of S. caninervis":"Exposure"}, axis=1)
df.head()#.append(df.sum().rename('Total')).assign(Total=lambda d: d.sum(1))

,Inferred Genetic Sex_F,Inferred Genetic Sex_M,Phenotypic Sex_F,Phenotypic Sex_M,Phenotypic Sex_S,Exposure
Coordinates,,,,,,
T1 105,10,0,1.0,0.0,9.0,I
T1 135,10,0,0.0,0.0,10.0,I
T1 150,7,0,0.0,0.0,10.0,I
T1 165,7,0,0.0,0.0,10.0,I
T1 180,9,0,0.0,0.0,10.0,I


# Error Correction

In [31]:
df = df[["Inferred Genetic Sex_F","Inferred Genetic Sex_M","Phenotypic Sex_F","Phenotypic Sex_M","Phenotypic Sex_S","Exposure"]]
df

,Inferred Genetic Sex_F,Inferred Genetic Sex_M,Phenotypic Sex_F,Phenotypic Sex_M,Phenotypic Sex_S,Exposure
Coordinates,,,,,,
T1 105,10,0,1.0,0.0,9.0,I
T1 135,10,0,0.0,0.0,10.0,I
T1 150,7,0,0.0,0.0,10.0,I
T1 165,7,0,0.0,0.0,10.0,I
T1 180,9,0,0.0,0.0,10.0,I
T1 195,10,0,0.0,0.0,10.0,I
T1 210,4,0,0.0,0.0,10.0,I
T1 240,10,0,4.0,0.0,6.0,I
T1 30,10,0,1.0,0.0,9.0,I


In [32]:
df.at["T2 105", "Phenotypic Sex_F"] = 0
df.at["T2 105", "Phenotypic Sex_M"] = 0
df.at["T2 105", "Phenotypic Sex_S"] = 10
df.at["T2 105", "Exposure"] = "O"
df

,Inferred Genetic Sex_F,Inferred Genetic Sex_M,Phenotypic Sex_F,Phenotypic Sex_M,Phenotypic Sex_S,Exposure
Coordinates,,,,,,
T1 105,10,0,1.0,0.0,9.0,I
T1 135,10,0,0.0,0.0,10.0,I
T1 150,7,0,0.0,0.0,10.0,I
T1 165,7,0,0.0,0.0,10.0,I
T1 180,9,0,0.0,0.0,10.0,I
T1 195,10,0,0.0,0.0,10.0,I
T1 210,4,0,0.0,0.0,10.0,I
T1 240,10,0,4.0,0.0,6.0,I
T1 30,10,0,1.0,0.0,9.0,I


In [141]:
df = df.dropna()
len(df.index)

40

# Tables for Jenna

In [37]:
exposure = df.groupby('Exposure').sum()[['Inferred Genetic Sex_F','Inferred Genetic Sex_M','Phenotypic Sex_F','Phenotypic Sex_M','Phenotypic Sex_S']]
exposure

,Inferred Genetic Sex_F,Inferred Genetic Sex_M,Phenotypic Sex_F,Phenotypic Sex_M,Phenotypic Sex_S
Exposure,,,,,
I,138,5,23,5,192
O,79,7,2,0,128
U,30,2,4,1,45


In [144]:
melted = pd.melt(exposure.reset_index(), id_vars=['Exposure'])
melted

,Exposure,variable,value
0,I,Inferred Genetic Sex_F,138
1,O,Inferred Genetic Sex_F,79
2,U,Inferred Genetic Sex_F,30
3,I,Inferred Genetic Sex_M,5
4,O,Inferred Genetic Sex_M,7
5,U,Inferred Genetic Sex_M,2
6,I,Phenotypic Sex_F,23
7,O,Phenotypic Sex_F,2
8,U,Phenotypic Sex_F,4
9,I,Phenotypic Sex_M,5


In [145]:
melted['Sex'] = melted['variable'].str[-1:]
melted['Expressing Sex'] = melted['variable'].str[0:1] == 'I'
analysis = melted.drop('variable', axis=1).rename({"Sex":"Sex","Exposure":"Exposure","Expressing Sex":"Expressing Sex","value":"Count"}, axis=1)
analysis = analysis[["Sex","Exposure","Expressing Sex","Count"]]
analysis.sort_values("Sex")


,Sex,Exposure,Expressing Sex,Count
0,F,I,True,138
1,F,O,True,79
2,F,U,True,30
6,F,I,False,23
7,F,O,False,2
8,F,U,False,4
3,M,I,True,5
4,M,O,True,7
5,M,U,True,2
9,M,I,False,5


In [42]:
#Table 1. Counts of phenotypic and genotypic females, males, and non-expressing ramets.
template = analysis.groupby(["Sex", "Expressing Sex"]).sum().reset_index()
grouped = template.groupby("Sex").agg(list)
grouped["Genotypic Count"] = list(template[template['Expressing Sex'] == True]['Count']) + [0]
grouped["Phenotypic Count"] = list(template[template['Expressing Sex'] == False]['Count'])
grouped_template = grouped[["Genotypic Count","Phenotypic Count"]]
grouped_template = grouped_template.append(grouped_template.sum().rename('Total')).assign(Total=lambda d: d.sum(1))
grouped_template['Genotypic Count %'] = 100 * grouped_template["Genotypic Count"]/261
grouped_template['Phenotypic Count %'] = 100 * grouped_template["Phenotypic Count"]/400
grouped_template['Total %'] = 100 * grouped_template["Total"]/661
grouped_template.to_csv("table1.csv")
grouped_template

,Genotypic Count,Phenotypic Count,Total,Genotypic Count %,Phenotypic Count %,Total %
Sex,,,,,,
F,247,29,276,95,7,42
M,14,6,20,5,2,3
S,0,365,365,0,91,55
Total,261,400,661,100,100,100


In [50]:
#Table 1 Proposed chi2 contingency
chi2_contingency(grouped.loc[["F","M"],["Genotypic Count","Phenotypic Count"]])

(5.055219691741432,
 0.024551869252984537,
 1,
 array([[243.36486486,  32.63513514],
        [ 17.63513514,   2.36486486]]))

In [69]:
#Table 2. Genotypic sex of non-expressing ramets in entirely non-expressing populations.
df_2 = df[df["Phenotypic Sex_S"] == 10].dropna()
entirely_female = len(df_2[df_2["Inferred Genetic Sex_M"] == 0]["Inferred Genetic Sex_F"])
entirely_male = len(df_2[df_2["Inferred Genetic Sex_F"] == 0]["Inferred Genetic Sex_M"])
mixed_df_2 = df_2[(df_2["Inferred Genetic Sex_F"] != 0) & (df_2["Inferred Genetic Sex_M"] != 0)]
mixed = len(mixed_df_2["Inferred Genetic Sex_F"] + mixed_df_2["Inferred Genetic Sex_M"])
ls_2 = np.array([entirely_female, entirely_male, mixed]).astype(int)
ls_2

array([22,  0,  4])

In [84]:
#Table 3. Genotypic sex of non-expressing ramets in populations that contain only phenotypic females and non-expressing ramets.
no_steriles = df[df["Phenotypic Sex_S"]!=10]
no_phen_males = no_steriles[no_steriles["Phenotypic Sex_M"] == 0].dropna()
entirely_female_npm = len(no_phen_males[no_phen_males["Inferred Genetic Sex_M"] == 0]["Inferred Genetic Sex_F"])
entirely_male_npm = len(no_phen_males[no_phen_males["Inferred Genetic Sex_F"] == 0]["Inferred Genetic Sex_M"])
mixed_no_phen_males = no_phen_males[(no_phen_males["Inferred Genetic Sex_F"] != 0) & (no_phen_males["Inferred Genetic Sex_M"] != 0)]
mixed_npm = len(mixed_no_phen_males["Inferred Genetic Sex_F"] + mixed_no_phen_males["Inferred Genetic Sex_M"])
ls_3 = [entirely_female_npm, entirely_male_npm, mixed_npm]
ls_3

[11, 1, 0]

In [85]:
#Table 4. Genotypic sex of non-expressing ramets in populations that contain only phenotypic males and non-expressing ramets.
no_steriles = df[df["Phenotypic Sex_S"]!=10]
no_phen_females = no_steriles[no_steriles["Phenotypic Sex_F"] == 0].dropna()
entirely_female_npf = len(no_phen_females[no_phen_females["Inferred Genetic Sex_M"] == 0]["Inferred Genetic Sex_F"])
entirely_male_npf = len(no_phen_females[no_phen_females["Inferred Genetic Sex_F"] == 0]["Inferred Genetic Sex_M"])
mixed_no_phen_females = no_phen_females[(no_phen_females["Inferred Genetic Sex_F"] != 0) & (no_phen_females["Inferred Genetic Sex_M"] != 0)]
mixed_npf = len(mixed_no_phen_females["Inferred Genetic Sex_F"] + mixed_no_phen_females["Inferred Genetic Sex_M"])
ls_4 = [entirely_female_npf, entirely_male_npf, mixed_npf]
ls_4


[0, 1, 1]

In [102]:
#Table 2-4 Chi2 Contingency Possible Global P-Value
p_values = pd.DataFrame({"Population Sex":["Entirely female", "Entirely male", "Mixed"],"Entirely non-expressing":ls_2, "Females and Non-Expressing":ls_3, "Males and Non-Expressing":ls_4}).set_index("Population Sex")
p_pivot = pd.pivot_table(p_values, values=["Entirely non-expressing", "Females and Non-Expressing", "Males and Non-Expressing"], index=["Population Sex"])
chi2_contingency(p_pivot)[1]

0.003981536456784525

In [157]:
pivoted = pd.DataFrame(p_values.T.reset_index().to_dict())
melted = pivoted.melt(id_vars='index')
melted["Pheno males present"] = melted['index'] == "Males and Non-Expressing"
melted["Pheno females present"] = melted['index'] == "Females and Non-Expressing"
melted = melted[["Pheno males present", "Pheno females present", "variable", "value"]]
melted = melted.rename({"variable":"Genetic pop make-up", "value":"Count"}, axis=1)
melted

,Pheno males present,Pheno females present,Genetic pop make-up,Count
0,False,False,Entirely female,22
1,False,True,Entirely female,11
2,True,False,Entirely female,0
3,False,False,Entirely male,0
4,False,True,Entirely male,1
5,True,False,Entirely male,1
6,False,False,Mixed,4
7,False,True,Mixed,0
8,True,False,Mixed,1


In [127]:
table_2_thru_4 = pd.pivot_table(melted, values="Count", index=["Genetic pop make-up","Pheno males present", "Pheno females present"])
table_2_thru_4

Count
Genetic pop make-up Pheno males present Pheno females present       
Entirely female     False               False                     22
                                        True                      11
                    True                False                      0
Entirely male       False               False                      0
                                        True                       1
                    True                False                      1
Mixed               False               False                      4
                                        True                       0
                    True                False                      1

In [137]:
versus_genetic = pd.pivot_table(melted, values="Count", index="Genetic pop make-up", columns=["Pheno males present", "Pheno females present"])
versus_males = pd.pivot_table(melted, values="Count", columns="Pheno males present", index=["Genetic pop make-up", "Pheno females present"])
versus_females = pd.pivot_table(melted, values="Count", columns="Pheno females present", index=["Genetic pop make-up","Pheno males present"])
versus_males


Pheno males present                        False  True 
Genetic pop make-up Pheno females present              
Entirely female     False                     22      0
                    True                      11    nan
Entirely male       False                      0      1
                    True                       1    nan
Mixed               False                      4      1
                    True                       0    nan

In [142]:
versus_genetic

Pheno males present   False       True 
Pheno females present False True  False
Genetic pop make-up                    
Entirely female          22    11     0
Entirely male             0     1     1
Mixed                     4     0     1

In [136]:
chi2_contingency(versus_genetic)

(15.376068376068377, 0.003981536456784525, 4, array([[21.45,  9.9 ,  1.65],
        [ 1.3 ,  0.6 ,  0.1 ],
        [ 3.25,  1.5 ,  0.25]]))

In [146]:
analysis

,Sex,Exposure,Expressing Sex,Count
0,F,I,True,138
1,F,O,True,79
2,F,U,True,30
3,M,I,True,5
4,M,O,True,7
5,M,U,True,2
6,F,I,False,23
7,F,O,False,2
8,F,U,False,4
9,M,I,False,5


In [147]:
#Table 5 Female
chi2_contingency([
    [23, 138],
    [2,79],
    [4,30]
])

(8.067273295841673,
 0.017709808365847558,
 2,
 array([[ 16.91666667, 144.08333333],
        [  8.51086957,  72.48913043],
        [  3.57246377,  30.42753623]]))

In [152]:
sum([23, 138] +
    [2,79] +
    [4,30])

276

In [149]:
#Table 5 Male
chi2_contingency([
    [5,5],
    [0,7],
    [1,2]
])


(4.92063492063492, 0.08540783306532157, 2, array([[3. , 7. ],
        [2.1, 4.9],
        [0.9, 2.1]]))

In [150]:
#Table 5 Total
chi2_contingency([
    [143,28],
    [86,2],
    [32,5]
])


(11.196931839642065,
 0.0037035408894207584,
 2,
 array([[150.78040541,  20.21959459],
        [ 77.59459459,  10.40540541],
        [ 32.625     ,   4.375     ]]))

# Prob Test

In [160]:
df

,Inferred Genetic Sex_F,Inferred Genetic Sex_M,Phenotypic Sex_F,Phenotypic Sex_M,Phenotypic Sex_S,Exposure,Total Pheno Pop
Coordinates,,,,,,,
T1 105,10,0,1,0,9,I,10
T1 135,10,0,0,0,10,I,10
T1 150,7,0,0,0,10,I,10
T1 165,7,0,0,0,10,I,10
T1 180,9,0,0,0,10,I,10
T1 195,10,0,0,0,10,I,10
T1 210,4,0,0,0,10,I,10
T1 240,10,0,4,0,6,I,10
T1 30,10,0,1,0,9,I,10


In [163]:
pop_type = []
for idx, row in df.iterrows():
    if row['Phenotypic Sex_S'] == 10:
        pop_type += ['Entirely non-expressing']
    else:
        if row['Phenotypic Sex_F'] == 0:
            pop_type += ['Phenotypic males present']
        elif row['Phenotypic Sex_M'] == 0:
            pop_type += ['Phenotypic females present']
        else:
            pop_type += ['Mixed']

In [164]:
df['Population Type'] = pop_type
df

,Inferred Genetic Sex_F,Inferred Genetic Sex_M,Phenotypic Sex_F,Phenotypic Sex_M,Phenotypic Sex_S,Exposure,Total Pheno Pop,Population Type
Coordinates,,,,,,,,
T1 105,10,0,1,0,9,I,10,Phenotypic females present
T1 135,10,0,0,0,10,I,10,Entirely non-expressing
T1 150,7,0,0,0,10,I,10,Entirely non-expressing
T1 165,7,0,0,0,10,I,10,Entirely non-expressing
T1 180,9,0,0,0,10,I,10,Entirely non-expressing
T1 195,10,0,0,0,10,I,10,Entirely non-expressing
T1 210,4,0,0,0,10,I,10,Entirely non-expressing
T1 240,10,0,4,0,6,I,10,Phenotypic females present
T1 30,10,0,1,0,9,I,10,Phenotypic females present


In [184]:
prob_table = df.groupby("Population Type").sum()[['Inferred Genetic Sex_F','Inferred Genetic Sex_M']]
m = list(prob_table['Inferred Genetic Sex_M'].astype(float))
f = list(prob_table['Inferred Genetic Sex_F'].astype(float))
prob_table

,Inferred Genetic Sex_F,Inferred Genetic Sex_M
Population Type,,
Entirely non-expressing,162,7
Phenotypic females present,82,3
Phenotypic males present,3,4


In [187]:
male_probs = dict(zip(prob_table.index, [mi/(mi+fi) for mi,fi in list(zip(m,f))]))
male_probs

{'Entirely non-expressing': 0.04142011834319527,
 'Phenotypic females present': 0.03529411764705882,
 'Phenotypic males present': 0.5714285714285714}

In [188]:
female_probs = dict(zip(prob_table.index, [fi/(mi+fi) for mi,fi in list(zip(m,f))]))
female_probs

{'Entirely non-expressing': 0.9585798816568047,
 'Phenotypic females present': 0.9647058823529412,
 'Phenotypic males present': 0.42857142857142855}

In [191]:
exp_table = df.groupby('Exposure').sum()[['Inferred Genetic Sex_F','Inferred Genetic Sex_M']]
exp_table

,Inferred Genetic Sex_F,Inferred Genetic Sex_M
Exposure,,
I,138,5
O,79,7
U,30,2


In [194]:
m = list(exp_table['Inferred Genetic Sex_M'].astype(float))
f = list(exp_table['Inferred Genetic Sex_F'].astype(float))
male_probs = dict(zip(exp_table.index, [mi/(mi+fi) for mi,fi in list(zip(m,f))]))
female_probs = dict(zip(exp_table.index, [fi/(mi+fi) for mi,fi in list(zip(m,f))]))
male_probs, female_probs

({'I': 0.03496503496503497, 'O': 0.08139534883720931, 'U': 0.0625},
 {'I': 0.965034965034965, 'O': 0.9186046511627907, 'U': 0.9375})